In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
from scipy.spatial import distance
import pickle
import json
import h5py
import pandas as pd


cuda = torch.device('cuda')
frame_size = 10
# https://drive.google.com/open?id=1kTyu05ZmtP2MA33J5hWdX8OyUYEDW4iI
movie_ref = pickle.load(open('../data/infos_pca128.pytorch', 'rb'))
# on my system os error often pops up
# try to open the file explorer and copy paste the path again 
# https://drive.google.com/open?id=1pPf-7AmUVceVfgfmKEJ6ireEDKEJHw-7
f = h5py.File("../data/static_ml20m.hdf5", "r")
# https://drive.google.com/open?id=1xjVI4uVQGsQ7tjOJ3594ZXmAEC_6yX0e
meta = json.load(open('../data/metadata.json'))

In [3]:
class Actor(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=2e-1):
        super(Actor, self).__init__()
        
        self.drop_layer = nn.Dropout(p=0.5)
        
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, num_actions)
        
        self.linear3.weight.data.uniform_(-init_w, init_w)
        self.linear3.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
        # state = self.state_rep(state)
        x = F.relu(self.linear1(state))
        x = self.drop_layer(x)
        x = F.relu(self.linear2(x))
        x = self.drop_layer(x)
        # x = torch.tanh(self.linear3(x)) # in case embeds are -1 1 normalized
        x = self.linear3(x) # in case embeds are standard scaled / wiped using PCA whitening
        # return state, x
        return x

In [4]:
ddpg = Actor(1290, 128, 256).to(cuda)
td3 = Actor(1290, 128, 256).to(cuda)
ddpg.load_state_dict(torch.load('../models/DDPG/ddpg_policy.pt'))
td3.load_state_dict(torch.load('../models/TD3/td3_policy_1.pt'))

batch_size = 5000
n_batches = (f['done'].shape[0] // batch_size) - 1

In [5]:
test_batch = [f[key][n_batches*batch_size:(n_batches+1)*batch_size] for key in
                 ['movies', 'ratings', 'done']]


movies, ratings, done = test_batch
movies, ratings, done = [torch.tensor(i.astype('float32')) for i in [movies, ratings, done]]
movies_tensor = torch.stack([torch.stack([movie_ref[int(i)] for i in ts]) for ts in movies])
    
state = torch.cat([movies_tensor[:, :-1, :].view(batch_size, -1),
                       ratings[:, :-1]], 1).to(cuda)


In [6]:
genres_dict = {'Animation': 16,
 'Comedy': 35,
 'Family': 10751,
 'Adventure': 12,
 'Fantasy': 14,
 'Romance': 10749,
 'Drama': 18,
 'Action': 28,
 'Crime': 80,
 'Thriller': 53,
 'Horror': 27,
 'History': 36,
 'Science Fiction': 878,
 'Mystery': 9648,
 'War': 10752,
 'Music': 10402,
 'Documentary': 99,
 'Western': 37,
 'TV Movie': 10770,
 'Sport': 1,
 'Sci-Fi': 878,
 'Musical': 10402,
 'Short': 2,
 'News': 3,
 'Film-Noir': 4}
genres_dict = {v: k for k, v in genres_dict.items()}

In [7]:
def rank(gen_action, metric):
    scores = []
    for i in movie_ref.keys():
        scores.append([i, metric(movie_ref[i], gen_action)])
    scores = list(sorted(scores, key = lambda x: x[1]))
    scores = scores[:10]
    ids = [i[0] for i in scores]
    for i in range(10):
        scores[i].extend([meta[str(scores[i][0])][key]  for key in ['title',
                                'genres', 'original_language', 'release_year', 'vote_average']])
        scores[i][3] = ' '.join([genres_dict[i] for i in scores[i][3]])

    indexes = ['id', 'score', 'title', 'genres', 'language', 'year', 'rating']
    table_dict = dict([(key,[i[idx] for i in scores]) for idx, key in enumerate(indexes)])
    table = pd.DataFrame(table_dict)
    return table

# DDPG

In [8]:
ddpg_action = ddpg(state)
# pick random action
ddpg_action = ddpg_action[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [9]:
from pandas.plotting import table
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(ddpg_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,102980,13.287885,Opera Jawa,Drama Musical Romance,jv,2006,5.3
1,26147,13.463657,A Thousand Clowns,Comedy Drama Romance,en,1965,7.2
2,65738,13.536163,Revenge of the Nerds III: The Next Generation,Comedy,en,1992,4.1
3,1574,13.573964,Fall,Drama Romance,en,1997,7.0
4,108953,13.611587,Crave,Drama Thriller,en,2012,4.7
5,128634,13.621246,Invisible Ghost,Horror Mystery Thriller,en,1941,5.3
6,122290,13.667379,Homeboy,Drama,en,1988,4.5
7,87996,13.689593,The Waiting Room,Drama,tr,2004,6.3
8,6336,13.738419,Marooned in Iraq,Drama,fa,2002,8.5
9,34314,13.754783,Funny Ha Ha,Drama Comedy,en,2002,6.2


In [10]:
rank(ddpg_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,112938,0.666276,The Detective,Drama Action Thriller Mystery,cn,2007,5.8
1,93702,0.682156,Tropical Fish,Comedy Drama,zh,1995,7.0
2,102980,0.691136,Opera Jawa,Drama Musical Romance,jv,2006,5.3
3,1891,0.691199,The Ugly,Horror Thriller,fr,1997,6.1
4,2881,0.693195,Double Jeopardy,Crime Drama Mystery Thriller,en,1999,6.3
5,88061,0.694654,Transistor Love Story,Musical Comedy Drama Romance,th,2001,6.8
6,80777,0.695098,Tere Naam,Romance Thriller,hi,2003,6.1
7,4118,0.700035,Hot Pursuit,Comedy,en,1987,6.4
8,81270,0.700478,My Soul to Take,Drama Horror Mystery Thriller,en,2010,5.4
9,117017,0.707220,The Wicker Tree,Mystery Horror Thriller,en,2011,4.4


In [11]:
rank(ddpg_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,112938,0.668885,The Detective,Drama Action Thriller Mystery,cn,2007,5.8
1,1891,0.679756,The Ugly,Horror Thriller,fr,1997,6.1
2,93702,0.682027,Tropical Fish,Comedy Drama,zh,1995,7.0
3,102980,0.691870,Opera Jawa,Drama Musical Romance,jv,2006,5.3
4,2881,0.692427,Double Jeopardy,Crime Drama Mystery Thriller,en,1999,6.3
5,88061,0.694414,Transistor Love Story,Musical Comedy Drama Romance,th,2001,6.8
6,81270,0.694985,My Soul to Take,Drama Horror Mystery Thriller,en,2010,5.4
7,80777,0.697562,Tere Naam,Romance Thriller,hi,2003,6.1
8,4118,0.702984,Hot Pursuit,Comedy,en,1987,6.4
9,3944,0.707520,Bootmen,Comedy Drama Musical,en,2000,5.8


In [12]:
rank(ddpg_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,79228,77.854828,Like Minds,Mystery Thriller Crime Horror,en,2006,6.5
1,81270,78.349391,My Soul to Take,Drama Horror Mystery Thriller,en,2010,5.4
2,112905,79.340487,Goliyon Ki Raasleela Ram-Leela,Drama Musical Romance,hi,2013,7.2
3,85397,79.463510,Red Riding Hood,Fantasy Thriller Horror,en,2011,5.9
4,27515,79.567648,Balzac and the Little Chinese Seamstress,Drama,zh,2002,7.2
5,60110,79.684758,Tom Sawyer,Family Musical,en,1973,6.2
6,129520,79.699777,Papa,Comedy Drama,en,2005,6.0
7,120486,79.742477,Darr,Drama Romance Thriller,hi,1993,7.8
8,90574,79.813952,Nora's Will,Drama Action Comedy Thriller,es,2008,7.3
9,121873,79.819755,The Man in Possession,Comedy,en,1931,7.0


In [13]:
rank(ddpg_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,102980,13.287885,Opera Jawa,Drama Musical Romance,jv,2006,5.3
1,26147,13.463657,A Thousand Clowns,Comedy Drama Romance,en,1965,7.2
2,65738,13.536163,Revenge of the Nerds III: The Next Generation,Comedy,en,1992,4.1
3,1574,13.573964,Fall,Drama Romance,en,1997,7.0
4,108953,13.611587,Crave,Drama Thriller,en,2012,4.7
5,128634,13.621246,Invisible Ghost,Horror Mystery Thriller,en,1941,5.3
6,122290,13.667379,Homeboy,Drama,en,1988,4.5
7,87996,13.689593,The Waiting Room,Drama,tr,2004,6.3
8,6336,13.738419,Marooned in Iraq,Drama,fa,2002,8.5
9,34314,13.754783,Funny Ha Ha,Drama Comedy,en,2002,6.2


In [14]:
rank(ddpg_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,98949,2.773865,The Eleventh Hour,Action Thriller,en,2008,4.4
1,87166,2.804971,Stolen,Crime Drama Mystery Thriller,en,2009,6.2
2,8255,2.815531,Chaos,Drama Mystery Thriller,ja,2000,6.2
3,94801,2.819741,The Steel Trap,Thriller,en,1952,6.0
4,95695,2.822823,Five Minarets in New York,Drama,en,2010,5.7
5,81676,2.844684,United,Comedy Drama,no,2003,5.4
6,8956,2.847730,Enduring Love,Drama Thriller Mystery,en,2004,5.8
7,43538,2.870498,The Mystery of Rampo,Drama Thriller Romance,ja,1995,7.0
8,810,2.875179,Kazaam,Fantasy Comedy Family,en,1996,3.8
9,5412,2.875501,The Villain,Comedy Western,en,1979,5.6


In [15]:
rank(ddpg_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,80777,0.709620,Tere Naam,Romance Thriller,hi,2003,6.1
1,45036,0.720816,The Chess Players,Drama,hi,1977,7.3
2,81270,0.724967,My Soul to Take,Drama Horror Mystery Thriller,en,2010,5.4
3,106859,0.725413,Amen,Fantasy Musical Romance,ml,2013,7.7
4,112938,0.727185,The Detective,Drama Action Thriller Mystery,cn,2007,5.8
5,117498,0.727500,Fandry,Drama Family,mr,2013,6.3
6,79846,0.729059,Spider Forest,Thriller Horror,ko,2004,6.6
7,93702,0.730386,Tropical Fish,Comedy Drama,zh,1995,7.0
8,2900,0.732566,Monkey Shines,Thriller Drama Horror Sci-Fi,en,1988,6.2
9,117017,0.735345,The Wicker Tree,Mystery Horror Thriller,en,2011,4.4


In [16]:
rank(ddpg_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,128634,118.335800,Invisible Ghost,Horror Mystery Thriller,en,1941,5.3
1,26638,119.994629,Prison,Crime Drama Horror Thriller,en,1987,6.1
2,34314,120.063980,Funny Ha Ha,Drama Comedy,en,2002,6.2
3,90,120.131500,The Journey of August King,Drama,en,1995,6.7
4,125944,120.139427,Desire: The Vampire,TV Movie Crime Horror Thriller,en,1982,4.0
5,117354,121.571365,Parasomnia,Horror Thriller,en,2008,4.6
6,7086,121.745369,Pygmalion,Drama Comedy,en,1938,7.6
7,82013,121.883720,Adrift,Drama,vi,2009,7.1
8,90106,122.248070,Pigskin Parade,Comedy Musical,en,1936,5.0
9,102980,122.681564,Opera Jawa,Drama Musical Romance,jv,2006,5.3


# TD3

In [17]:
td3_action = td3(state)
# pick random action
td3_action = td3_action[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [18]:
from pandas.plotting import table 
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(td3_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,44731,9.145951,Stay Alive,Horror Thriller,en,2006,5.4
1,130,9.189915,Angela,Drama,en,1995,5.5
2,103279,9.239122,What Maisie Knew,Drama,en,2013,7.1
3,101339,9.270727,Snowbeast,Horror TV Movie,en,1977,4.6
4,26402,9.334769,Harper Valley P.T.A.,Comedy,en,1978,7.0
5,470,9.334893,House Party 3,Comedy,en,1994,5.1
6,71285,9.345050,Beeswax,Drama,en,2009,6.4
7,103840,9.350063,Whisper of Sin,Drama,lt,2007,6.3
8,26180,9.458993,Up the Down Staircase,Drama,en,1967,7.7
9,26147,9.462359,A Thousand Clowns,Comedy Drama Romance,en,1965,7.2


In [19]:
rank(td3_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,5895,0.653973,Making Contact,Sci-Fi Fantasy Horror Adventure,en,1985,5.9
1,107065,0.655423,I-See-You.Com,Comedy,en,2006,4.6
2,217,0.661813,The Babysitter,Drama Thriller,en,1995,4.5
3,60825,0.668399,Mulberry Street,Horror,en,2006,5.4
4,101339,0.673493,Snowbeast,Horror TV Movie,en,1977,4.6
5,447,0.678424,The Favor,Comedy Romance,en,1994,4.3
6,5018,0.681310,Motorama,Comedy Adventure,en,1991,6.5
7,470,0.681352,House Party 3,Comedy,en,1994,5.1
8,118348,0.683478,Trance,Thriller TV Movie,en,2001,6.0
9,71285,0.685412,Beeswax,Drama,en,2009,6.4


In [20]:
rank(td3_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,107065,0.651935,I-See-You.Com,Comedy,en,2006,4.6
1,217,0.655472,The Babysitter,Drama Thriller,en,1995,4.5
2,447,0.656521,The Favor,Comedy Romance,en,1994,4.3
3,5895,0.657034,Making Contact,Sci-Fi Fantasy Horror Adventure,en,1985,5.9
4,60825,0.663391,Mulberry Street,Horror,en,2006,5.4
5,71285,0.683427,Beeswax,Drama,en,2009,6.4
6,101339,0.685087,Snowbeast,Horror TV Movie,en,1977,4.6
7,92376,0.686049,This Time Around,Comedy Romance,en,2003,5.6
8,5018,0.686492,Motorama,Comedy Adventure,en,1991,6.5
9,118348,0.687060,Trance,Thriller TV Movie,en,2001,6.0


In [21]:
rank(td3_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,1153,78.799513,Raw Deal,Crime Thriller,en,1948,7.1
1,80549,80.166563,Easy A,Comedy,en,2010,6.8
2,44903,80.182612,Dementia,Crime Horror,en,1955,6.5
3,131152,80.368781,The Fat Spy,Comedy,en,1966,2.5
4,179,80.762994,Mad Love,Drama Romance,en,1995,5.0
5,35957,81.062940,Red Eye,Horror Thriller,en,2005,6.2
6,112844,81.123510,Bus Stop,Comedy,en,1982,6.8
7,217,81.343714,The Babysitter,Drama Thriller,en,1995,4.5
8,101339,81.423641,Snowbeast,Horror TV Movie,en,1977,4.6
9,26357,81.430446,Trilogy of Terror,Horror Thriller TV Movie,en,1975,6.3


In [22]:
rank(td3_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,44731,9.145951,Stay Alive,Horror Thriller,en,2006,5.4
1,130,9.189915,Angela,Drama,en,1995,5.5
2,103279,9.239122,What Maisie Knew,Drama,en,2013,7.1
3,101339,9.270727,Snowbeast,Horror TV Movie,en,1977,4.6
4,26402,9.334769,Harper Valley P.T.A.,Comedy,en,1978,7.0
5,470,9.334893,House Party 3,Comedy,en,1994,5.1
6,71285,9.345050,Beeswax,Drama,en,2009,6.4
7,103840,9.350063,Whisper of Sin,Drama,lt,2007,6.3
8,26180,9.458993,Up the Down Staircase,Drama,en,1967,7.7
9,26147,9.462359,A Thousand Clowns,Comedy Drama Romance,en,1965,7.2


In [23]:
rank(td3_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,71285,1.976391,Beeswax,Drama,en,2009,6.4
1,3955,2.008488,The Ladies Man,Comedy,en,2000,5.3
2,112596,2.020457,Missing William,Drama,en,2014,3.5
3,74733,2.024548,Trap,Drama Thriller,en,2007,5.0
4,4765,2.029783,L.I.E.,Drama Crime,en,2001,6.9
5,118252,2.037397,All Cheerleaders Die,Horror Thriller Comedy,en,2013,5.1
6,33917,2.057688,The Member of the Wedding,Drama,en,1952,6.9
7,86139,2.080920,The Traveler,Drama,fa,1974,7.5
8,7310,2.086384,Raw Deal,Action,en,1986,5.6
9,1742,2.091486,Caught Up,Drama Thriller Crime,en,1998,4.8


In [24]:
rank(td3_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,217,0.687466,The Babysitter,Drama Thriller,en,1995,4.5
1,44903,0.688797,Dementia,Crime Horror,en,1955,6.5
2,101339,0.693471,Snowbeast,Horror TV Movie,en,1977,4.6
3,447,0.705373,The Favor,Comedy Romance,en,1994,4.3
4,131152,0.708161,The Fat Spy,Comedy,en,1966,2.5
5,103279,0.708845,What Maisie Knew,Drama,en,2013,7.1
6,107065,0.708908,I-See-You.Com,Comedy,en,2006,4.6
7,3177,0.714231,Next Friday,Comedy,en,2000,6.4
8,60825,0.715055,Mulberry Street,Horror,en,2006,5.4
9,470,0.719001,House Party 3,Comedy,en,1994,5.1


In [25]:
rank(td3_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,103279,80.269730,What Maisie Knew,Drama,en,2013,7.1
1,44731,81.254990,Stay Alive,Horror Thriller,en,2006,5.4
2,101339,81.335190,Snowbeast,Horror TV Movie,en,1977,4.6
3,26180,82.694778,Up the Down Staircase,Drama,en,1967,7.7
4,130,82.895546,Angela,Drama,en,1995,5.5
5,69988,84.193680,Humpday,Comedy,en,2009,5.4
6,26402,84.290314,Harper Valley P.T.A.,Comedy,en,1978,7.0
7,129707,84.567711,The Lazarus Effect,Thriller Horror,en,2015,5.1
8,129348,84.728485,A Short History of Decay,Comedy,en,2014,6.0
9,121413,84.786644,Picture This,Comedy,en,2008,5.6


# BCQ

In [26]:
class VAE(nn.Module):
    def __init__(self, state_dim, action_dim, latent_dim):
        super(VAE, self).__init__()
        #encoder
        self.e1 = nn.Linear(state_dim + action_dim, 750)
        self.e2 = nn.Linear(750, 750)

        self.mean = nn.Linear(750, latent_dim)
        self.log_std = nn.Linear(750, latent_dim)
        
        #decoder
        self.d1 = nn.Linear(state_dim + latent_dim, 750)
        self.d2 = nn.Linear(750, 750)
        self.d3 = nn.Linear(750, action_dim)
        
        self.latent_dim = latent_dim


    def forward(self, state, action):
        # z is encoded state + action
        z = F.relu(self.e1(torch.cat([state, action], 1)))
        z = F.relu(self.e2(z))

        mean = self.mean(z)
        # Clamped for numerical stability 
        log_std = self.log_std(z).clamp(-4, 15)
        std = torch.exp(log_std)
        z = mean + std * torch.FloatTensor(np.random.normal(0, 1, size=(std.size()))).to(cuda) 

        # u is decoded action
        u = self.decode(state, z)

        return u, mean, std
    
    def decode(self, state, z=None):
        # When sampling from the VAE, the latent vector is clipped to [-0.5, 0.5]
        if z is None:
            z = torch.FloatTensor(np.random.normal(0, 1, size=(state.size(0), self.latent_dim))).to(cuda).clamp(-0.5, 0.5)

        a = F.relu(self.d1(torch.cat([state, z], 1)))
        a = F.relu(self.d2(a))
        return self.d3(a)

In [27]:
class BCQ_Actor(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=3e-1):
        super(BCQ_Actor, self).__init__()
        
        self.drop_layer = nn.Dropout(p=0.5)
        
        self.linear1 = nn.Linear(num_inputs + num_actions, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, num_actions)
        
        self.linear3.weight.data.uniform_(-init_w, init_w)
        self.linear3.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state, action):
        a = torch.cat([state, action], 1)
        a = F.relu(self.linear1(a))
        a = self.drop_layer(a)
        a = F.relu(self.linear2(a))
        a = self.drop_layer(a)
        a = self.linear3(a) 
        return a + action

In [28]:
bcq_actor = BCQ_Actor(1290, 128, 256).to(cuda)
bcq_vae = VAE(1290, 128, 512).to(cuda)

bcq_actor.load_state_dict(torch.load('../models/BCQ/bcq_policy.pt'))
bcq_vae.load_state_dict(torch.load('../models/BCQ/bcq_vae.pt'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [29]:
#state_rep = torch.repeat_interleave(state, 50, 0)
#sampled_action = vae_net.decode(state_rep)
#perturbed_action = target_policy_net(state_rep, sampled_action)

sampled_actions = bcq_vae.decode(state)
perturbed_actions= bcq_actor(state, sampled_actions)

In [36]:
bcq_action = perturbed_actions[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [37]:
rank(bcq_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,4186,8.648272,The Fortune Cookie,Romance Comedy,en,1966,7.4
1,4256,8.696550,The Center of the World,Drama Romance,en,2001,5.3
2,115711,8.801120,TINY: A Story About Living Small,Documentary,en,2013,6.5
3,78574,9.010764,Winter's Bone,Drama,en,2010,6.9
4,106897,9.063599,Stuck Between Stations,Drama Romance,en,2011,6.2
5,109265,9.069264,Bickford Shmeckler's Cool Ideas,Comedy,en,2006,5.6
6,84242,9.151440,Welcome to the Rileys,Drama,en,2010,6.5
7,1436,9.170608,Falling in Love Again,Drama Romance,en,1980,6.1
8,25912,9.186786,The Enchanted Cottage,Drama Romance,en,1945,7.1
9,103269,9.258514,Unconditional,Drama,en,2012,6.8


In [38]:
rank(bcq_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,115711,0.618910,TINY: A Story About Living Small,Documentary,en,2013,6.5
1,87994,0.671117,Gilles' Wife,Drama,fr,2004,5.7
2,7745,0.679103,The Scent of Green Papaya,Drama Romance,vi,1993,7.4
3,7002,0.680505,Mindwalk,Drama,en,1990,6.9
4,7901,0.681659,The Haunted Palace,Horror,en,1963,6.5
5,4256,0.682728,The Center of the World,Drama Romance,en,2001,5.3
6,1012,0.686986,Old Yeller,Adventure Drama Western,en,1957,7.0
7,6420,0.691303,Night Passage,Western,en,1957,6.0
8,120390,0.692164,Natural Selection,Comedy Drama,en,2011,6.2
9,4186,0.692538,The Fortune Cookie,Romance Comedy,en,1966,7.4


In [39]:
rank(bcq_action, distance.correlation)

,id,score,title,genres,language,year,rating
0,115711,0.609512,TINY: A Story About Living Small,Documentary,en,2013,6.5
1,7745,0.675647,The Scent of Green Papaya,Drama Romance,vi,1993,7.4
2,87994,0.678899,Gilles' Wife,Drama,fr,2004,5.7
3,7002,0.681784,Mindwalk,Drama,en,1990,6.9
4,4186,0.683726,The Fortune Cookie,Romance Comedy,en,1966,7.4
5,6420,0.685657,Night Passage,Western,en,1957,6.0
6,4256,0.690369,The Center of the World,Drama Romance,en,2001,5.3
7,7901,0.690832,The Haunted Palace,Horror,en,1963,6.5
8,1012,0.694327,Old Yeller,Adventure Drama Western,en,1957,7.0
9,7976,0.695163,Ken Park,Drama,en,2002,5.8


In [40]:
rank(bcq_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,36056,77.838349,Man of the West,Action Western,en,1958,6.3
1,43813,77.932971,Broken Lance,Western,en,1954,6.0
2,60293,78.008085,The Wackness,Drama,en,2008,6.4
3,115711,78.090413,TINY: A Story About Living Small,Documentary,en,2013,6.5
4,8486,78.263328,Bachelor Mother,Comedy Romance,en,1939,6.6
5,6420,78.391936,Night Passage,Western,en,1957,6.0
6,55440,78.461566,The Final Season,Drama,en,2007,5.2
7,3095,78.735481,The Grapes of Wrath,Drama,en,1940,7.9
8,4373,78.756830,Pootie Tang,Action Comedy,en,2001,5.1
9,85790,78.977798,Meek's Cutoff,Drama Western,en,2010,6.5


In [41]:
rank(bcq_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,4186,8.648272,The Fortune Cookie,Romance Comedy,en,1966,7.4
1,4256,8.696550,The Center of the World,Drama Romance,en,2001,5.3
2,115711,8.801120,TINY: A Story About Living Small,Documentary,en,2013,6.5
3,78574,9.010764,Winter's Bone,Drama,en,2010,6.9
4,106897,9.063599,Stuck Between Stations,Drama Romance,en,2011,6.2
5,109265,9.069264,Bickford Shmeckler's Cool Ideas,Comedy,en,2006,5.6
6,84242,9.151440,Welcome to the Rileys,Drama,en,2010,6.5
7,1436,9.170608,Falling in Love Again,Drama Romance,en,1980,6.1
8,25912,9.186786,The Enchanted Cottage,Drama Romance,en,1945,7.1
9,103269,9.258514,Unconditional,Drama,en,2012,6.8


In [42]:
rank(bcq_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,114068,1.883288,Captivated: The Trials of Pamela Smart,Documentary,en,2014,6.5
1,109265,1.915959,Bickford Shmeckler's Cool Ideas,Comedy,en,2006,5.6
2,3588,1.916321,The King of Marvin Gardens,Drama,en,1972,6.3
3,5015,1.920499,Monster's Ball,Drama Romance,en,2001,6.7
4,83357,1.933282,A Gathering of Eagles,Drama Romance,en,1963,6.4
5,6532,1.939592,Up the Sandbox,Drama Comedy,en,1972,6.9
6,112650,1.954881,Watercolors,Drama Romance,en,2008,6.3
7,54254,1.959912,Come Early Morning,Drama Romance,en,2006,5.7
8,3916,1.965924,Remember the Titans,Drama,en,2000,7.6
9,90899,1.977351,The People vs. George Lucas,Documentary,en,2010,6.4


In [43]:
rank(bcq_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,115711,0.654988,TINY: A Story About Living Small,Documentary,en,2013,6.5
1,6420,0.695984,Night Passage,Western,en,1957,6.0
2,8711,0.700442,Mr. Blandings Builds His Dream House,Comedy Romance,en,1948,7.2
3,4256,0.700953,The Center of the World,Drama Romance,en,2001,5.3
4,36056,0.711548,Man of the West,Action Western,en,1958,6.3
5,4428,0.716119,The Misfits,Western Drama Action Romance,en,1961,6.9
6,70513,0.717385,West of the Divide,Action Western,en,1934,5.0
7,127873,0.717705,Jay Mohr: Funny for a Girl,Comedy,en,2012,3.0
8,60293,0.717726,The Wackness,Drama,en,2008,6.4
9,7976,0.718254,Ken Park,Drama,en,2002,5.8


In [44]:
rank(bcq_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,115711,77.869026,TINY: A Story About Living Small,Documentary,en,2013,6.5
1,4256,78.333496,The Center of the World,Drama Romance,en,2001,5.3
2,4186,79.347725,The Fortune Cookie,Romance Comedy,en,1966,7.4
3,106897,79.512337,Stuck Between Stations,Drama Romance,en,2011,6.2
4,26226,81.518318,"Hi, Mom!",Comedy Drama,en,1970,5.8
5,85072,82.361031,The Wild and Wonderful Whites of West Virginia,Documentary,en,2009,6.9
6,109265,82.850830,Bickford Shmeckler's Cool Ideas,Comedy,en,2006,5.6
7,103269,82.852074,Unconditional,Drama,en,2012,6.8
8,70889,83.050339,The Pom Pom Girls,Comedy,en,1976,5.4
9,25912,83.364594,The Enchanted Cottage,Drama Romance,en,1945,7.1
